In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import numpy as np
import os

In [ ]:
#getting pixel values of the images to create X array 
#y is the list of class names as folder names

image_dir = os.getcwd() + '\\gear_images_resized\\'
X = []
y = []
for image_path in os.listdir(os.fsencode(image_dir)):
    for image in os.listdir(os.fsencode(image_dir + str(image_path).replace("b'", '').replace("'", ''))):
        img_dir = os.fsencode(image_dir + str(image_path).replace("b'", '').replace("'", '') + '\\' + str(image).replace("b'", '').replace("'", ''))
        img = Image.open(img_dir)
        if img.size == (128,128):
            I = np.array(img.getdata(),
                        np.uint8).reshape(49152 , 1)

            X.append(I.reshape(1 , 49152))
            y.append(str(image_path).replace("b'", '').replace("'", ''))
        
        else:
            break

In [ ]:
 X = np.array(X).reshape(np.array(X).shape[0], 3, 128, 128) # 3 comes from RGB

In [ ]:
#normalization of data

X = X/255

In [ ]:
#categorical classes turned into encoded values

lb = LabelEncoder()

y = lb.fit_transform(y)
y = keras.utils.to_categorical(y,6) 

In [ ]:
#split data to train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
#create a sequential artificial deep network model

model = Sequential()

In [ ]:
# 64 is the number of output filters in the convolution
# (3,3) is the height and width of 2D convolution window
# relu activation function for nonlinearity

model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
## randomly close 0.30 of neurons to prevent overfitting during training
model.add(Dropout(0.3))
# add layer with 3x3 filters to model
model.add(MaxPooling2D(pool_size=(3, 3), dim_ordering="th"))
model.add(Dropout(0.10))

In [ ]:
model.add(Conv2D(32, (3, 3), data_format='channels_first'))
# add relu activation function
model.add(Activation('relu'))
model.add(Dropout(0.3))
# add 2x2 MaxPooling layer 
model.add(MaxPooling2D(pool_size=(2, 2)))
# randomly close 0.10 of neurons to prevent overfitting during training
model.add(Dropout(0.10))

In [ ]:
# convert 2D images to 1D vector
model.add(Flatten())
# add to model 512 neurons
model.add(Dense(256))
# add relu activation function
model.add(Activation('relu'))
# randomly close 0.30 of neurons
model.add(Dropout(0.3))
model.add(Dense(6)) # num_classes = 6
#add "Softmax" function to calculate the probability of classes
model.add(Activation('softmax'))

In [ ]:
opt = keras.optimizers.adam(lr=0.01, decay=1e-6)

In [ ]:
early_stopping_monitor = EarlyStopping(patience=3)

In [ ]:
model.compile(loss='binary_crossentropy', #binary crossentropy for 2 class problem (dog/cat)
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,
              batch_size=128,
              epochs=15,
              validation_data=(X_test, y_test),
              verbose=1,
              shuffle=True,
              )

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
model.save("model.h5")

In [ ]:
import flask
import keras.models
from werkzeug.wrappers import Request, Response
from flask import Flask
from flask import request
from keras.applications.imagenet_utils import decode_predictions

In [ ]:
# used flask to use our model as a web service
# the function returns class of the input image

app = Flask(__name__)

@app.route('/handle_request', methods=['GET', 'POST'])
def handle_request():
    
    img = Image.open("path/image.jpg")
    loaded_model = keras.models.load_model('model.h5')
    A = []
    
    I = np.array(img.getdata(),
                np.uint8).reshape(49152 , 1)
    A.append(I.reshape(1 , 49152))

    A = np.array(A).reshape(np.array(A).shape[0], 3, 128, 128)
    A = A/255

    preds = loaded_model.predict(A)
    y_classes = preds.argmax(axis=-1)

    return str(lb.inverse_transform(y_classes[0]))
    

In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)